In [ ]:
from sqlalchemy import create_engine, Column, Integer, Float, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import os
import time
import pandas as pd
from sklearn.ensemble import IsolationForest
from pyod.models.abod import ABOD
import datetime

Base = declarative_base()

class Anomaly(Base):
    __tablename__ = 'anomalies'
    id = Column(Integer, primary_key=True)
    file_name = Column(String)
    date_time = Column(DateTime)
    selected_model = Column(String)
    anomaly_score = Column(Float)

def monitor_folder(folder):
    engine = create_engine("sqlite:///anomalies.db")
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    session = Session()

    before = set(os.listdir(folder))
    while True:
        time.sleep(10) # sleep for 10 seconds
        after = set(os.listdir(folder))
        new_files = after - before
        if new_files:
            print("New files detected: ", new_files)
            for file in new_files:
                file_path = os.path.join(folder, file)
                try:
                    if file.endswith('.xls'):
                        data = pd.read_excel(file_path)
                        time_series = data['value'].values.reshape(-1,1)
                    else:
                        data = pd.read_csv(file_path)
                    time_series = data['value'].values.reshape(-1,1)
                    
                    # Isolation Forest
                    clf = IsolationForest(contamination='auto')
                    clf.fit(time_series)
                    scores_if = clf.decision_function(time_series)
                    
                    # Angle-Based Outlier Detector (ABOD)
                    clf = ABOD(contamination='auto')
                    clf.fit(time_series)
                    scores_abod = clf.decision_function(time_series)
                    
                    # Select the best model based on the mean of the anomaly scores
                    if abs(scores_if).mean() < abs(scores_abod).mean():
                        selected_model = "Isolation Forest"
                        selected_scores = scores_if
                    else:
                        selected_model = "ABOD"
                        selected_scores = scores_abod
                    
                    print("Selected Model:", selected_model)
                    
                    # Write anomaly scores to database
                    engine = create_engine('sqlite:///anomalies.db')
                    Session = sessionmaker(bind=engine)
                    session = Session()
                    
                    for score in selected_scores:
                        anomaly = AnomalyScore(file_name=file, model=selected_model, score=score)
                        session.add(anomaly)
                        
                    session.commit()
                except Exception as e:
                    print("Error processing file:", file, e)
            before = after

if __name__ == "__main__":
        folder = r"D:\Documents"
        monitor_folder(folder)

New files detected:  {'data.xls'}
Error processing file: data.xls 'str' object has no attribute 'eval'
